In [1]:
# 将数据预处理及特征工程的完整代码封装成一个函数
# 该函数实现即能处理训练数据集（train.csv） 也能处理新的数据集（new.csv）

In [1]:
import pandas as pd
def prepare_data(path,role):
    """
    功能：实现对训练数据或者新的数据的数据预处理工作
    参数：
        @param path 读取的数据文件路径 
        @param role 数据文件的用途 ,两个取值：
                    train（表明预处理的是训练集）
                    test （表明预处理的是新的数据集）
    Return:
        返回预处理好后的训练集或测试集
    """
    titanic_df = pd.read_csv(path)
    # 根据数据集角色不一样，删除数据列也不一样
    if role == "train":
        titanic_df = titanic_df.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
    else:
        # 保留PassengerId，方便将来预测结果和PassengerId对应
        titanic_df = titanic_df.drop(["Name","Ticket","Cabin"],axis=1)
    # 标记年龄缺失值
    titanic_df["AgeIsMissing"] = 0 
    titanic_df.loc[titanic_df["Age"].isnull(),"AgeIsMissing"] = 1
    # 平均值填充年龄
    age_mean = round(titanic_df.Age.mean())
    titanic_df.Age.fillna(age_mean,inplace=True)
    # 用频率出现最高的登船港口填充缺失值
    titanic_df.Embarked.fillna("S",inplace=True)
    # 年龄自定义分箱
    cut_points = [0,18,25,40,60,100]
    titanic_df["AgeBin"] = pd.cut(titanic_df.Age,bins=cut_points)
    # 船票价格等深分箱
    titanic_df["FareBin"] = pd.qcut(titanic_df.Fare,5)
    # 构造FamilySize变量
    titanic_df["FamilySize"] = titanic_df["SibSp"] + titanic_df["Parch"] + 1
    # 构造IsAlone变量
    titanic_df["IsAlone"] =0
    titanic_df.loc[titanic_df["FamilySize"] == 1,"IsAlone"] = 1
    # 构造IsMother变量
    titanic_df["IsMother"] = 0
    titanic_df.loc[(titanic_df["Sex"] == "female")&(titanic_df["Parch"] > 0)&(titanic_df["Age"]>20),
                   "IsMother"] = 1
    # 性别和年龄分箱特征进行组合，共组合出10种
    titanic_df["SexAgeCombo"] = titanic_df["Sex"] + "_" + titanic_df["AgeBin"].astype(str)
    # 一大堆独热编码
    Pclass_onehot = pd.get_dummies(titanic_df.Pclass,prefix="Pclass")
    Sex_onehot = pd.get_dummies(titanic_df.Sex,prefix="Sex") 
    Embarked_onehot = pd.get_dummies(titanic_df.Embarked,prefix="Embarked")
    AgeBin_onehot = pd.get_dummies(titanic_df.AgeBin,prefix="AgeBin")
    FareBin_onehot = pd.get_dummies(titanic_df.FareBin,prefix="FareBin")
    FamilySize_onehot = pd.get_dummies(titanic_df.FamilySize,prefix="FamilySize")
    SexAgeCombo_onehot = pd.get_dummies(titanic_df.SexAgeCombo,prefix="SexAgeCombo")
    # 数据集的角色不一样，拼接的数据列也不一样
    if role=="train":
        TrainData = pd.concat([titanic_df[["Survived","AgeIsMissing","IsAlone","IsMother"]],
                              Pclass_onehot,Sex_onehot,Embarked_onehot,AgeBin_onehot,
                              FareBin_onehot,FamilySize_onehot,SexAgeCombo_onehot],axis=1)
    else:
        TrainData = pd.concat([titanic_df[["PassengerId","AgeIsMissing","IsAlone","IsMother"]],
                              Pclass_onehot,Sex_onehot,Embarked_onehot,AgeBin_onehot,
                              FareBin_onehot,FamilySize_onehot,SexAgeCombo_onehot],axis=1)
    # 返回处理好的数据集
    return TrainData